In [3]:
import pandas as pd
import naive_bayes as nb
import numpy as np

In [4]:
import numpy as np
from collections import Counter
import operator

from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import operator

class NaiveBayesClassifier:
    def __init__(self, alpha = 1):
        self.dic_data_index = dict()
        self.dic_data_by_class = dict()
        self.dic_data_posterior = dict()
        self.dic_data_prob_fitur = dict()
        self.prior = dict()
        self.class_ = 0
        self.alpha = alpha
#         self.count_doc_c = dict()

    def train (self, X, y):
#         print(X)
        self.X = X
        self.y = y
        
        vectorizer = CountVectorizer(binary=True)
        self.X = vectorizer.fit_transform(X).A
        self.fitur = vectorizer.get_feature_names()
        self.len_fitur = len(self.fitur)
        self.y = np.array(self.y)
        self.class_ = sorted(set(y))
        
        self.prior_class = dict(zip(Counter(self.y).keys(), Counter(self.y).values()))
        self.count_doc_c = self.prior_class.copy()
        for key, value in self.prior_class.items():
#             self.count_doc_c.update({key:len(self.y)})
            self.prior_class.update({key:value/len(self.y)})
        
        #menghitung jumlah class
        self.len_data = len(self.y)

        #mencari index data untuk class tertentu
#         self.dic_data_index = dict()
        for i in self.class_:
            isi_list = list()
            for index, j in enumerate(self.y):
        #         print(j)
                if i == j:
                    isi_list.append(index)
            self.dic_data_index.update({i:isi_list})
            
#         self.dic_data_by_class = dict()
        for key, value in self.dic_data_index.items():
            self.dic_data_by_class.update({key:(self.X[value])})
        # del self.dic_data_index
        
        self.count_fitur_c = dict()
        self.dic_data_prob_fitur = dict()
        for c, value in self.dic_data_by_class.items():
            count_per_fitur = list()
            prob_fitur = list()
            sum_value  = value.sum()
            for ix, f in enumerate(value.transpose()):
                prob_fitur.append((f.sum()+self.alpha)/(sum_value+self.len_fitur))
                count_per_fitur.append(f.sum())
#                 prob_fitur.append((f.sum()/(len(value)+self.len_fitur)))
#                 print(c,str(f.sum()), "+",str(self.alpha),"/", str(len(value)),"+", str(self.len_fitur), self.fitur[ix])
            
            _dict_count_fitur = dict(zip(self.fitur, count_per_doc))
            bobot_fitur = dict(zip(self.fitur,prob_fitur))
            
            self.count_fitur_c.update({c:_dict_count_fitur})
            self.dic_data_prob_fitur.update({c:bobot_fitur})
    
    def predict(self, X):
#         tostr = lamda str(x):x
        if self.alpha <= 0:
            raise Exception("alpha tidak boleh kurang dari atau sama dengan 0, alpha="+str(self.alpha)) 
        X = sorted(set(X.split()))
#         try:
        self.dict_predict = dict()
        for c, value in self.dic_data_prob_fitur.items():
#             print(len(value))

            bobot_fitur = list()
            for kata in X:
#                 print(kata)
                if kata in value:
#                     print(value[kata])
                    # print(c,kata,value[kata])
                    bobot_fitur.append((self.count_fitur_c[c][kata]+self.alpha)/(self.count_doc_c[c]+self.len_fitur))
                    print("P({}|{})=({}+{})/({}+{})"
                    .format(kata,c,self.count_fitur_c[c][kata],self.alpha,self.count_doc_c[c],self.len_fitur), end=" = ")
                    print((self.count_fitur_c[c][kata]+self.alpha)/(self.count_doc_c[c]+self.len_fitur))
#  
#                     print("P"+"("+kata+"|"+c+")="+str((value[kata]+(self.alpha/(self.count_doc_c[c]+self.len_fitur)))))
#                     
            if len(bobot_fitur)>0:
                post_prior = np.prod(bobot_fitur)*self.prior_class[c]
#                 print(type(bobot_fitur))
                bobot_fitur_ = [str(round(x, 3)) for x in bobot_fitur]
                print("P({}|{}) = {} x {} = {}".format(c,"X",self.prior_class[c], " x ".join(bobot_fitur_), post_prior,20))
#                 print(bobot_fitur)
                self.dict_predict.update({c:post_prior})
            print("")
        return max(self.dict_predict.items(), key=operator.itemgetter(1))[0]
#         except:
#             print("err404or")
#             return max(self.prior_class.items(), key=operator.itemgetter(1))[0]

In [5]:
foo = pd.read_excel("Book1.xlsx")

In [6]:
label = np.array(foo.label.tolist())
komentar = np.array(foo.komentar.tolist())

In [7]:
komentar

array(['obat herbal sehat murah',
       'ingin putih glowing langsing ideal konsultasi langsung',
       'mau punya kulit putih harga murah add putih kulit',
       'mantap terima kasih pak',
       'Tetap satu pintu Pak nanti Dana Siluman',
       'Ini mantul mantap betul', 'sehat terus pak'], dtype='<U54')

In [8]:
model = NaiveBayesClassifier()

In [10]:
# model.alpha = 1
# model.train(komentar, label)

In [33]:
model.predict("obat herbal langsing sehat")

P(herbal|non spam)=(0+1)/(4+30) = 0.029411764705882353
P(langsing|non spam)=(0+1)/(4+30) = 0.029411764705882353
P(obat|non spam)=(0+1)/(4+30) = 0.029411764705882353
P(sehat|non spam)=(1+1)/(4+30) = 0.058823529411764705
P(non spam|X) = 0.5714285714285714 x 0.029 x 0.029 x 0.029 x 0.059 = 8.552169086645446e-07

P(herbal|spam)=(1+1)/(3+30) = 0.06060606060606061
P(langsing|spam)=(1+1)/(3+30) = 0.06060606060606061
P(obat|spam)=(1+1)/(3+30) = 0.06060606060606061
P(sehat|spam)=(1+1)/(3+30) = 0.06060606060606061
P(spam|X) = 0.42857142857142855 x 0.061 x 0.061 x 0.061 x 0.061 = 5.782124489863032e-06



'spam'

In [25]:
def data_separate(y, complement=False):
    kelas = sorted(set(y))
    
    dict_index = dict()
    for c in kelas:
        index = list()
        for ix, c_ in enumerate(y):
            if complement==False and c==c_:
                index.append(ix)
            elif complement==True and c!=c_:
                index.append(ix)
        dict_index.update({c:index})
    return dict_index

def prior_(y):
    unik = sorted(set(y))
    dict_p = dict()
    for c in unik:
        count = y.tolist().count(c)
        dict_p.update({c:count/len(y)})
    return dict_p 

class MultinominalNaiveBayes:
    def __init__(self, alpha=1):
        self.alpha = alpha
        self.dict_nb = 0
        
    def train(self, X, y):
        self.class_ = sorted(set(y))
        self.prior = prior_(y)
        index_data = data_separate(y)
        self.an = len(X.A[0])
        self.X = X
        
        self.dict_nb = dict()
        for c in self.class_:
            n_yi = np.sum(self.X[index_data[c]].A, axis=0)
            n_y = self.X[index_data[c]].A.sum()
            self.dict_nb.update({c:{}})
            self.dict_nb[c]["n_y"] = n_y
            self.dict_nb[c]["n_yi"] = n_yi
            
    def predict(self, X):
        self.X = X
        result = list()
        for i in self.X.A:
            list_pst = list()
            for c in self.class_:
                hat_theta = (self.dict_nb[c]["n_yi"]+self.alpha)/(self.dict_nb[c]["n_y"]+self.an)
                posterior = np.prod(hat_theta**i)*self.prior[c]
                list_pst.append(posterior)
            result.append(self.class_[list_pst.index(max(list_pst))])
        return result


In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer as w
# from sklearn.feature_extraction.text import CountVectorizer as w
corpus = komentar
vectorizer = w()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())

print(X.shape)

['add', 'betul', 'dana', 'glowing', 'harga', 'herbal', 'ideal', 'ingin', 'ini', 'kasih', 'konsultasi', 'kulit', 'langsing', 'langsung', 'mantap', 'mantul', 'mau', 'murah', 'nanti', 'obat', 'pak', 'pintu', 'punya', 'putih', 'satu', 'sehat', 'siluman', 'terima', 'terus', 'tetap']
(7, 30)


In [36]:
y = np.array(['a','a','b','b'])

In [39]:
# model = MultinominalNaiveBayes()
model.train(X,label)
print(model.alpha)
model.predict(X)

1


['spam', 'spam', 'spam', 'non spam', 'non spam', 'non spam', 'non spam']

In [38]:
model.alpha=1

In [412]:
def data_separate(y, complement=False):
    kelas = sorted(set(y))
    
    dict_index = dict()
    for c in kelas:
        index = list()
        for ix, c_ in enumerate(y):
            if complement==False and c==c_:
                index.append(ix)
            elif complement==True and c!=c_:
                index.append(ix)
        dict_index.update({c:index})
    return dict_index

def prior_(y):
    unik = sorted(set(y))
    dict_p = dict()
    for c in unik:
        count = y.tolist().count(c)
        dict_p.update({c:count/len(y)})
    return dict_p 

class CNaiveBayes:
    def __init__(self, alpha=1):
        self.alpha = alpha
        self.dict_nb = 0
        
    def train(self, X, y):
        self.class_ = sorted(set(y))
        self.prior = prior_(y)
        index_data = data_separate(y, complement=True) #complement=False
        self.an = len(X.A[0])
        self.X = X
        
        self.dict_nb = dict()
        
        for c in self.class_:
            self.n_yi = np.sum(self.X[index_data[c]].A, axis=0)
            self.n_y = self.X[index_data[c]].A.sum()
            self.dict_nb.update({c:{}})
            hat_theta = (self.n_yi+self.alpha)/(self.n_y+self.an)
            w_ci = np.log(hat_theta)
            
            abs_sum_wci= np.sum(np.abs(w_ci))
            norm_wci = w_ci/abs_sum_wci
            self.dict_nb[c]['w_ci'] = norm_wci
            self.dict_nb[c]["n_y"] = self.n_y
            
    def set_alpha(self, alpha =1):
        self.alpha = alpha
#         self.dict_nb = dict()
        for c in self.class_:
            self.dict_nb.update({c:{}})
            hat_theta = (self.n_yi+self.alpha)/(self.n_y+self.an)
            w_ci = np.log(hat_theta)
            
            abs_sum_wci= np.sum(np.abs(w_ci))
            norm_wci = w_ci/abs_sum_wci
            self.dict_nb[c]['w_ci'] = norm_wci
#             self.dict_nb[c]["n_y"] = self.n_y
            
    def predict(self, X):
        try:
            self.X = X
            result = list()
            for i in self.X.A:
                list_pst = list()
                for c in self.class_:
                    x = self.dict_nb[c]["w_ci"]
                    posterior = np.sum(x*i)
                    list_pst.append(posterior)
                result.append(self.class_[list_pst.index(min(list_pst))])
            return result
        except:
            return [self.class_[0]]

In [418]:
# model = CNaiveBayes()
model.train(X,label)
print(model.alpha)
model.predict(X[2])

10


['spam']

In [417]:
model.set_alpha(alpha =10)
model.alpha

10

In [394]:
a= np.sum(X[[1,2]].A, axis=0)
print(a)#/
sum_ = (X[[1,2]].A.sum())
print(sum_)
print(a/sum_)

[1 0 0 1 1 0 1 1 0 0 1 2 1 1 0 0 1 1 0 0 0 0 1 3 0 0 0 0 0 0]
16
[0.0625 0.     0.     0.0625 0.0625 0.     0.0625 0.0625 0.     0.
 0.0625 0.125  0.0625 0.0625 0.     0.     0.0625 0.0625 0.     0.
 0.     0.     0.0625 0.1875 0.     0.     0.     0.     0.     0.    ]


In [64]:
1/16

0.0625

In [141]:
np.array([3,5])**np.array([2,2])

array([ 9, 25])

In [31]:
print(X[dataaa['non spam']])

  (0, 20)	0.3971064382343257
  (0, 9)	0.5596754170004217
  (0, 27)	0.5596754170004217
  (0, 14)	0.4645786606417216
  (1, 26)	0.3921287418123664
  (1, 2)	0.3921287418123664
  (1, 18)	0.3921287418123664
  (1, 21)	0.3921287418123664
  (1, 24)	0.3921287418123664
  (1, 29)	0.3921287418123664
  (1, 20)	0.2782270638667323
  (2, 1)	0.5206467559864713
  (2, 15)	0.5206467559864713
  (2, 8)	0.5206467559864713
  (2, 14)	0.43218152024617124
  (3, 28)	0.6753558276781849
  (3, 20)	0.4791851475403949
  (3, 25)	0.5606033360566132


In [27]:
dataaa

{'non spam': [3, 4, 5, 6], 'spam': [0, 1, 2]}

In [103]:
0==0.0

True

In [209]:
0**1

0

In [303]:
(1+5)/(2+3)

1.2

In [304]:
1/(2+3)

0.2

In [306]:
5/(2+3)

1.0

In [330]:
np.log(0)

/home/lufias/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


-inf

In [ ]:
def data_separate(y, complement=False):
    kelas = sorted(set(y))
    
    dict_index = dict()
    for c in kelas:
        index = list()
        for ix, c_ in enumerate(y):
            if complement==False and c==c_:
                index.append(ix)
            elif complement==True and c!=c_:
                index.append(ix)
        dict_index.update({c:index})
    return dict_index

def prior_(y):
    unik = sorted(set(y))
    dict_p = dict()
    for c in unik:
        count = y.tolist().count(c)
        dict_p.update({c:count/len(y)})
    return dict_p 

from sklearn.feature_extraction.text import CountVectorizer

class NaiveBayesC:
    def __init__(self, alpha=1):
        self.alpha = alpha
        self.dict_nb = 0
        
    def train(self, X, y):
        
        vectorizer = CountVectorizer(binary=True)
        self.X = vectorizer.fit_transform(X).A
        
        self.class_ = sorted(set(y))
        self.prior = prior_(y)
        index_data = data_separate(y)
        self.an = len(X.A[0])
        
        self.dict_nb = dict()
        for c in self.class_:
            n_yi = np.sum(self.X[index_data[c]].A, axis=0)
#             n_y = self.X[index_data[c]].A.sum()
            n_y = len(self.X[index_data[c]].A)
            self.dict_nb.update({c:{}})
            self.dict_nb[c]['hat_theta'] = n_yi/(n_y+self.an)
            self.dict_nb[c]["n_y"] = n_y
            
    def predict(self, X):
        self.X = X
        result = list()
        for i in self.X.A:
            list_pst = list()
            for c in self.class_:
                laplace = self.alpha/(self.dict_nb[c]["n_y"]+self.an)
                x = self.dict_nb[c]["hat_theta"]+laplace
                posterior = np.prod(x**i)*self.prior[c]
                list_pst.append(posterior)
            result.append(self.class_[list_pst.index(max(list_pst))])
        return result